In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import seaborn as sns
from gerrychain import Graph, GeographicPartition, Partition, Election, accept
from gerrychain.updaters import Tally, cut_edges
import glob
import functools
import operator

In [11]:
def foldl(func, acc, xs):
  return functools.reduce(func, xs, acc)

foldr = lambda func, acc, xs: functools.reduce(lambda x, y: func(y, x), xs[::-1], acc)

In [24]:
def get_state_runs(seats, iters=100000, pop_bal=2.0, min_col="nWVAP",
                   ls=[2,5,10,20,40,80]):
    results = {}

    for l in ls:
        short_burst = glob.glob("data/states/SC_dists{}_{}opt_{}%_100000_sbl{}_score0_*.npy".format(seats, 
                                                                                                min_col, pop_bal, l))
        results[str(l)] = np.zeros((len(short_burst), iters))
        for i, run in enumerate(short_burst):
            results[str(l)][i] = np.load(run).flatten()
    return results

In [25]:
def create_state_df(runs, iters=100000):
    df_st = pd.DataFrame()
    for l in runs.keys():
        for i in range(runs[l].shape[0]):
            df = pd.DataFrame()
            df["Step"] = np.arange(iters)
            df["Maximum"] = np.maximum.accumulate(runs[l][i].flatten())
            df["run-type"] = "Short Burst" if float(l) > 1 else "Biased Run" if float(l) < 1 else "Unbiased Run"
            df["param"] = "b = {}".format(l) if float(l) > 1 else "q = {}".format(l)
            df_st = df_st.append(df, ignore_index=True)
    return df_st

In [26]:
la_runs_bvap = get_state_runs(124, ls=[2,5,10,25,50,100,200], 
                              pop_bal=4.5, min_col="BVAP")

In [27]:
ubs = glob.glob("data/unbiased_SC/SC_dists124_4.5%_100000_unbiased_*.p")
ub_runs = {}
for i, run in enumerate(ubs):
     with open(run, "rb") as f:
        ub_runs[i] = pickle.load(f)

In [28]:
runs = list(ub_runs.values())
bvap = foldl(lambda x,y: np.concatenate((x,y["BVAP"]), axis=0), runs[0]["BVAP"], runs[1:])

IndexError: list index out of range